# main.py
It serves as the entry point of the project Milestone 2 and contains the code to initialize and coordinate
the execution of other modules or classes.

It call methods that
    Connect to source and download data
    Clean the data
    Upload the data to a Postgresql database on localhost named sales_data

Those methods are contained in three classes coded in the files

- DatabaseConnector ---> database_utils.py
- DataCleaning --> data_cleaning.py
- DataExtractor --> data_extraction.py

In [1]:
# import the DatabaseConnector class for connecting and uploading data from various sources
from database_utils import DatabaseConnector as database_connector
# import the data cleaning Class
from data_cleaning import DataCleaning as data_cleaner
# import the DataExtractor class
from data_extraction import DataExtractor as data_extractor

Read the connection credential for the SQL databased

In [11]:
# file_name_psql should contain the yaml file with the credentials
file_name_psql = 'postgresql_creds.yaml'
cred_dict_psql = database_connector.read_db_creds(file_name_psql)
print(cred_dict_psql)

{'HOST': 'localhost', 'USER': 'postgres', 'PASSWORD': 'macchio', 'DATABASE': 'sales_data_test01', 'PORT': 5432}


## User data

In [7]:
# the credata_extractorntials to connect the RDS AWS server are stored in this yaml file
file_name = 'db_creds.yaml'

# read credata_extractorntial and transform them from yaml to dict
cred_dict = database_connector.read_db_creds(file_name)

# initialize the SQLalchemy engine
RDS_engine = database_connector.init_db_engine(cred_dict)

# get the list of tables in the RDS database
tables_list = data_extractor.list_db_tables(RDS_engine)

# print the list of tables
print(tables_list)

# read the data from the RDS table of users and stores in the users_data variable
users_data = data_extractor.read_RDS_table(tables_list[1], RDS_engine)


# clean the user data via the relative method
user_data_clean = data_cleaner.clean_user_data(users_data)

# upload the user data to the SQL server
database_connector.upload_to_db(user_data_clean, 'dim_users', cred_dict_psql)

SQL engine successfully connected
['legacy_store_details', 'legacy_users', 'orders_table']
{'HOST': 'localhost', 'USER': 'postgres', 'PASSWORD': 'macchio', 'DATABASE': 'sales_data_test01', 'PORT': 5432}


## Card data

In [12]:
# extract the card data
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
card_data = data_extractor.retrieve_pdf_data(pdf_link)

# clean the data through the relative method
card_data_clean = data_cleaner.clean_card_data(card_data)

# upload the card data to the SQL server
database_connector.upload_to_db(card_data_clean, 'dim_card_details', cred_dict_psql)

## Stores data

In [14]:
# key and url to access the number of stores 
key = {'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
url = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'

# retrieve the number of stores
number_of_stores = data_extractor.list_number_of_stores(url, key)

# print the number of stores
print("the number of stores to be extracted is: ", number_of_stores)

# url for the stores information
base_url = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'

# download the information for each store
stores_df = data_extractor.retrieve_stores_data(base_url, number_of_stores, key)

# clean the stores data
stores_data_clean = data_cleaner.clean_store_data(stores_df)

# upload the stores data to the SQL database
database_connector.upload_to_db(stores_data_clean, 'dim_store_details', cred_dict_psql)
print('upload successful')

the number of stores to be extracted is:  451
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
26

## Product data

In [ ]:
# log into AWS CLI with (i) Access Key ID and (ii) Secret Access Key
import subprocess

# Pause script execution
input("Press Enter to pause the script and log in to AWC CLI")

# Log into AWS CLI
subprocess.call(["aws", "configure"])

# extract the products data from the s3 server
products_df = data_extractor.extract_from_s3('s3://data-handling-public/', filename = 'products.csv')

# clean the data
products_df_clean = data_cleaner.clean_product_data(products_df)
products_df_clean_converted_units = data_cleaner.clean_product_weights(products_df_clean)

# upload to the SQL server
database_connector.upload_to_db(products_df_clean_converted_units, 'dim_products', cred_dict_psql)

## Orders data

In [16]:
orders_data = data_extractor.read_RDS_table(tables_list[2], RDS_engine)
orders_data_clean = data_cleaner.clean_orders_data(orders_data)

database_connector.upload_to_db(orders_data_clean, 'orders_table', cred_dict_psql)

url = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'
date_details = data_extractor.download_date_details(url)
print('date_details_downloaded')
date_details_clean = data_cleaner.clean_date_details(date_details)

database_connector.upload_to_db(date_details_clean, 'dim_date_times', cred_dict_psql)

date_details_downloaded
cleaning date details
